In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

### Let's review how we will use our dependencies.
We'll use Pandas to read our CSV file and create the locations and measurements from the DataFrame. We'll use gmaps and the API key to create heatmaps and the locations map, and we'll use the requests dependency to make a request to the Google Places JSON file. This will allow us to get hotel locations from the latitude and longitude of the city.

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Barrow,US,2021-04-21 18:09:42,71.2906,-156.7887,17.6,79,75,8.05
1,1,Punta Arenas,CL,2021-04-21 18:09:43,-53.1500,-70.9167,46.4,57,0,8.05
2,2,Arraial Do Cabo,BR,2021-04-21 18:09:43,-22.9661,-42.0278,78.8,78,40,14.97
3,3,Longyearbyen,SJ,2021-04-21 18:09:16,78.2186,15.6401,24.8,68,75,11.50
4,4,Clyde River,CA,2021-04-21 18:08:00,70.4692,-68.5914,10.4,92,75,11.50


In [3]:
# Gmaps needs coordinates as a float or integer, so we check the DF column types
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

## Google heatmaps do not plot negative numbers. 

If you have a maximum temperature that is less than 0 °F, then you will get an `InvalidWeightException` error for this line of code:

`heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)`

To remove the negative temperatures we can use a `for` loop to iterate through the `max_temp` and add the temperatures that are greater than 0 °F to a new list.

## `For` loop option for removing temps lower than 0 and setting to 0 degrees

#### Don't use

In [5]:
# Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]
# temps = []
# for temp in max_temp:
#     temps.append(max(temp, 0))

### create the heatmap for the maximum temperature.
General syntax is below:

#### 1. Assign the locations to an array of latitude and longitude pairs.
`locations = [latitude, longitude]`

#### 2. Assign the weights variable to some values.
`temperatures = # an array of length equal to the locations array length`

#### 3. Assign the figure variable to the gmaps.figure() attribute.
`fig = gmaps.figure()`

#### 4. Assign the heatmap_layer variable to the heatmap_layer attribute and add in the locations.
`heatmap_layer = gmaps.heatmap_layer(locations, weights=temperatures)`

#### 5. Add the heatmap layer.
`fig.add_layer(heatmap_layer)`

#### 6. Call the figure to plot the data.
`fig`

In [6]:
# Heatmap of temperature
# Get the latitude and longitude.
# locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
# max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
# fig = gmaps.figure()

# Assign the heatmap variable.
# heat_layer = gmaps.heatmap_layer(locations, weights=temps)

# Add the heatmap layer.
# fig.add_layer(heat_layer)

# Call the figure to plot the data.
# fig

## Use list comprehension instead
`heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])`

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable.
fig = gmaps.figure()

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Adjust Heatmap Zoom, Intensity, and Point Radius

Using `gmaps.figure()` attribute

In [9]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable and format
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Tweak gmaps.heat_layer() options
If you review the gmaps documentation, you may notice there is a dissipation option for creating heatmaps that can be added to the `gmaps.heat_layer()` attribute.

The options for this are:

1. The default option for the dissipation is "True," so we need to set our "dissipation" to "False."
2. We can add `max_intensity` to make each measurement have a better gradient variance.
3. We can add `point_radius` to make each measurement radius larger.

Also, you'll have to tweak these values until you are satisfied with the final map.

Let's modify our `gmaps.heat_layer()` attribute to `heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)` and rerun our code.

## Heatmap for max temperature

In [16]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

# Assign the figure variable and format
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Assign the heatmap variable.
# Tweak max_intensity and point_radius attributes to alter the gradient thresholds and sizes for a better looking map
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=170, point_radius=3)

# Add the heatmap layer.
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Heatmap for percent HUmidity

In [23]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Heatmap for percent cloudiness

In [26]:
# Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=200, point_radius=3)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Heatmap for wind speed

In [32]:
# Heatmap of wind speed
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=40, point_radius=3)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Create user input prompt for Temperature preferences

In [33]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Filter dataset based on user inputs

In [43]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Arraial Do Cabo,BR,2021-04-21 18:09:43,-22.9661,-42.0278,78.80,78,40,14.97
6,6,Mawlaik,MM,2021-04-21 18:09:44,23.6333,94.4167,76.05,33,0,1.07
10,10,Bombay,IN,2021-04-21 18:06:54,19.0144,72.8479,84.20,79,99,4.61
12,12,Lorengau,PG,2021-04-21 18:09:46,-2.0226,147.2712,81.91,80,98,7.40
15,15,Kieta,PG,2021-04-21 18:09:47,-6.2167,155.6333,78.69,82,78,1.48
16,16,Karratha,AU,2021-04-21 18:09:47,-20.7377,116.8463,78.31,52,1,8.55
18,18,Avarua,CK,2021-04-21 18:09:39,-21.2078,-159.7750,77.00,78,40,11.50
22,22,Kattivakkam,IN,2021-04-21 18:09:49,13.2167,80.3167,82.40,83,20,8.05
23,23,Rikitea,PF,2021-04-21 18:09:27,-23.1203,-134.9692,77.99,71,40,9.89
30,30,Eyl,SO,2021-04-21 18:09:40,7.9803,49.8164,83.37,74,60,8.97


In [44]:
# Check to see if new preferred_cities_df has any null values
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       2
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

### Shows country column has null values

so we remove values using df.dropna() function

In [45]:
preferred_cities_df = preferred_cities_df.dropna()

In [46]:
# Confirm no null values
preferred_cities_df.count()

City_ID       180
City          180
Country       180
Date          180
Lat           180
Lng           180
Max Temp      180
Humidity      180
Cloudiness    180
Wind Speed    180
dtype: int64

## Get Travel Destinations
Pull in hotel info into new DF

Don't add the hotel information to the `preferred_cities_df` DataFrame because this DataFrame is our filtered DataFrame, and the customer will always filter it for each trip. We'll need to create a new DataFrame specifically for the data needed to create a heatmap and pop-up markers.

Make a copy of the `preferred_cities_df` DataFrame and name it `hotel_df`. For the `hotel_df`, keep the columns **"City," "Country," "Max Temp," "Lat," and "Lng."** Add a new column to the `hotel_df` DataFrame to hold the name of the hotel.

In [47]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# Create column to populate hotel names from coordinates data
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,78.80,-22.9661,-42.0278,
6,Mawlaik,MM,76.05,23.6333,94.4167,
10,Bombay,IN,84.20,19.0144,72.8479,
12,Lorengau,PG,81.91,-2.0226,147.2712,
15,Kieta,PG,78.69,-6.2167,155.6333,
16,Karratha,AU,78.31,-20.7377,116.8463,
18,Avarua,CK,77.00,-21.2078,-159.7750,
22,Kattivakkam,IN,82.40,13.2167,80.3167,
23,Rikitea,PF,77.99,-23.1203,-134.9692,
30,Eyl,SO,83.37,7.9803,49.8164,


### Retrieve Hotels from a Nearby Search

#### Set the Parameters for a Nearby Search
https://developers.google.com/maps/documentation/places/web-service/search#PlaceSearchRequests%0D%0A

The Nearby Search request lets us perform these tasks:

1. Search for places within a specified area.
2. Refine the search using keywords and specifying the type of place we are searching for.
3. Use an API URL, where the output can be either XML or JSON format.

To pull info we'll add a specified area and keyword to search by providing specific parameters

Specific parameters include:

1. Our API key
2. A location, which will be the latitude and longitude
3. A radius for the search. The radius can be up to 50,000 meters or approximately 31 miles. This distance is much too great for finding hotels, so we'll keep our search to 5,000 meters.
4. If we use the `rankby=distance` for a parameter, we need to add one or more of the three parameters above. We won't use the `rankby=distance` because we will be search based on the radius from a given latitude and longitude.


For our hotel search, we'll use these parameters:

1. API key
2. Latitude and longitude
3. 5,000-meter radius
4. Type of place: https://developers.google.com/places/web-service/supported_types

On the webpage, Table 1 shows all the different Place types values. "Hotel" does not appear, but "lodging" does, so we will use the string "lodging" for the type parameter.

We can use the same format to make a request with the Google Places API. The following base URL is provided to retrieve the JSON format of the data:

https://maps.googleapis.com/maps/api/place/nearbysearch/json

#### Using requests.get() using parameters in the URL

We can add the parameters as a dictionary of strings, using the `params` keyword argument `requests.get('base URL', params=parameters)`

### Iterate Through hotel_df DataFrame
We can use the `iterrows()` function to perform the iteration; however, we need to provide the `index` and the `row` in the `for` loop using this syntax: `for index, row in df.iterrows()`.


#### Need to add try-except block, so any location point not close enough to a "hotel" doesn't cause an error
Because we ended up with a `IndexError` we have to create an exceptoion for that specific error and any others that apply.

In [52]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name.
    ### NOT NEEDED WITH TRY/EXCEPT BELOW
    # hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")
        
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [53]:
# Check hotel_df for populated data
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,78.80,-22.9661,-42.0278,Pousada Porto Praia
6,Mawlaik,MM,76.05,23.6333,94.4167,AKZ Guest House
10,Bombay,IN,84.20,19.0144,72.8479,Hotel Avon Ruby-Dadar
12,Lorengau,PG,81.91,-2.0226,147.2712,Lorengau Harbourside Hotel
15,Kieta,PG,78.69,-6.2167,155.6333,Uruna Bay Retreat
16,Karratha,AU,78.31,-20.7377,116.8463,ibis Styles Karratha
18,Avarua,CK,77.00,-21.2078,-159.7750,Paradise Inn
22,Kattivakkam,IN,82.40,13.2167,80.3167,Raj's villa
23,Rikitea,PF,77.99,-23.1203,-134.9692,Pension Maro'i
30,Eyl,SO,83.37,7.9803,49.8164,Amiin Hotel


## Create a Maximum Temperature Heatmap from a Hotel DataFrame

### Using `weights = max_temp`

In [60]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Using `weights = [max(temp, 0) for temp in max_temp]` list comprehension verison

In [61]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

## Adding google maps markers hotel data
https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#markers-and-symbols

`markers = gmaps.marker_layer(marker_locations)`, where the `marker_locations` are latitudes and longitudes

In [62]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

### Add a pop-up marker for each city that displays the hotel name, city name, country, and maximum temperature.

we need to add an `info_box_template` that has the following syntax

`info_box_template = """`

`<dl>`

`<dt>Name</dt><dd>{column1}</dd>`

`<dt>Another name</dt><dd>{column2}</dd>`

`</dl>`

`"""`

**First**, the `info_box_template` variable is assigned to a multiline string using three quotes. The text inside the multiline string is HTML code. HTML code is defined by the opening and closing the angular brackets (e.g., `<tag>` and `<tag/>`). Angular brackets always come in pairs. The opening angular bracket is followed by some text inside, such as `dl`, `dt`, and `dd`. The closing angular bracket is preceded by a forward-slash ("/"). The text inside with the angular brackets is called a **tag**. We'll see more examples of HTML code in an upcoming module.

Here's what these tags mean:

- The `<dl>` tag is a description list (dl).
- The `<dt>` tag is a term or name in a description list that is nested under the `<dl>` tag.
- The `<dd>` tag is used to define the term or name or `<dt>` tag.

If we were to write out these tags on paper, it would look like this.

- Description List: `<dl>`
    - Description Term: `<dt>`
        - Description Definition: `<dd>`
    - Description Term: `<dt>`
        - Description Definition: `<dd>`

For our purposes, we'll add the hotel name, city name, country code, and the maximum temperature values from the `hotel_df` DataFrame as the description definition


**Next**, add the data to the code by iterating through the `hotel_df` DataFrame using the `iterrows()` function. Then add the information to the `gmaps.marker_layer()` attribute with the locations.

According to the documentation (https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#markers-and-symbols), we can use this example to create the data for the `info_box_template`:

From the documentation, we can take this code:

`plant_info = [info_box_template.format(**plant) for plant in nuclear_power_plants]`

Then edit this code by replacing "`plant_info`" with "`hotel_info`," and then use the `iterrows()` function to get the index and data in the row to add to the marker.

In [65]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

### Let's review what we're doing with this code.

1. We set the `hotel_info` equal to the `info_box_content`.
2. In the list comprehension, `info_box_template.format(**row) for index, row in hotel_df.iterrows()`, we iterate through each "row" of the `hotel_df` DataFrame and then format the `info_box_template` with the data we set to populate the from each row. Remember, we are not using every row; we are only using the rows defined in the `info_box_template`, which are `Hotel Name`, `City`, `Country`, and `Max Temp`.

### create the heatmap with markers

add `info_box_content=hotel_info` to the `gmaps.marker_layer()` attribute with the locations. 

#### Our final cell should look like the following:

In [66]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp],dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))